# pytorch 

|　フレームワーク | 主な特徴 |
|------------| ----------- |
|TensorFlow, Keras | 産業向け |
| PyTorch, Chainer | 研究向け（カスタマイズ性が高い） | 

## PyTorchの構成

| 構成内容 | 説明 |
|---------|-----|
|torch | メインのネームスペースでTensorや様々な数学関数がこのパッケージに含まれる。NumPyの構造を模している|
| torch.autograd | 自動微分のための関数が含まれる。自動微分のon/offを制御するコンテキストマネージャのenable_grad/no_gradや独自の微分可能関数を定義する際に使用する基底クラスであるFunctionなどが含まれる | 
| torch.nn | ニューラルネットワークを構築するための様々なデータ構造やレイヤーが定義されている。例えばConvolutionやLSTM、ReLUなどの活性化関数やMSELossなどの損失関数も含まれる |
| torch.optim | 確率的勾配降下（SGD）を中心としたパラメータ最適化アルゴリズムが実装されている | 
| torch.utils.data | SGDの繰り返し計算を回す際のミニバッチを作るためのユーティリティ関数が含まれている |
| torch.onnx | 	ONNX(Open Neural Network Exchange)1の形式でモデルをエクスポートするために使用する。 | 

## データの操作
PyTorch には、データを操作するための 2 つのプリミティブがあります: torch.utils.data.DataLoaderとtorch.utils.data.Dataset. Datasetサンプルとそれに対応するラベルを格納しDataLoader、 iterable で をラップしDatasetを作成します。

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

PyTorch は、 TorchText、 TorchVision、TorchAudioなどのドメイン固有のライブラリを提供し、そのすべてにデータセットが含まれています。
今回はTorchVision データセットを使用します。

このtorchvision.datasetsモジュールにはDataset、CIFAR、COCO などの多くの実世界のビジョン データのオブジェクトが含まれています。今回はFashionMNIST データセットを使用します。
すべての TorchVisionDatasetには 2 つの引数が含まれています

In [ ]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

Datasetを引数として DataLoaderに渡します。
これは、データセットを iterable でラップし、自動バッチ処理、サンプリング、シャッフル、マルチプロセス データ ロードをサポートします。ここでは、64 のバッチ サイズを定義します。
つまり、データローダー iterable の各要素は、64 個の機能とラベルのバッチを返します。

In [ ]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

PyTorch のデータのロード について更に読んでください。

## モデルを作成する

PyTorch でニューラルネットワークを定義するため、nn.Module から継承したクラスを作成します。__init__ 関数でネットワーク層を定義して forward 関数でデータがどのようにネットワークを通り抜けるか指定します。ニューラルネットワークの演算を高速化するためには、利用可能であればそれを GPU に移します。

In [ ]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

## モデル・パラメータの最適化
モデルを訓練するためには、損失関数と optimizer を必要とします。

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

単一訓練ループ内で、モデルは訓練データセット (バッチでそれに供給されます) 上で予測を行ない、モデルのパラメータを調整するために予測誤差を逆伝播します。

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

モデルが学習していることを確認するためにテスト・データセットに対してそのパフォーマンスも確認します。

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

訓練プロセスは幾つかの反復 (エポック) に渡り実行されます。各エポックの間に、モデルはより良い予測を行なうためにパラメータを学習します。各エポックでモデルの精度と損失をプリントします ; 総てのエポックで精度が増加して損失が減少することを見たいです。

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

## モデルをセーブする
モデルをセーブする一般的な方法は (モデルパラメータを含む) 内部状態辞書をシリアライズすることです。

In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

## モデルをロードする
モデルをロードするためのプロセスはモデル構造の再作成と状態辞書のそれへのロードを含みます。

In [ ]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

このモデルは今では予測を行なうために利用できます。

In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')
